In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py

PATH = '/storage/home/adz6/group/project'
VNADATAPATH = os.path.join(PATH, 'datasets/data/lab/vna')
DAQDATAPATH = os.path.join(PATH, 'datasets/data/lab/dig')

VNARESULTS = os.path.join(PATH, 'results/moth/vna')
VNACORRECTIONS = os.path.join(PATH, 'results/moth/vna_correction_files')

LABDATAPATH = os.path.join(PATH, 'labdata')
#MOTHPATH = os.path.join(PATH, 'moth')
#MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
#PLOTPATH = os.path.join(PATH, 'plots/moth')
#MOTHRESULTS = os.path.join(PATH, 'results/moth')


def CalculateMagPhase(h5data, name, nch=120):
    
    new_h5file = h5py.File(name, 'w')
    
    for i, key_i in enumerate(h5data):
        new_h5file.create_group(key_i)
        print(key_i)
        for j, key_j in enumerate(h5data[key_i]):
            if key_j == 'array':
                phase = np.unwrap(np.angle(h5data[key_i][key_j][:, 0:nch, :]), axis=1)
                mag = abs(datafile[key_i][key_j][:, 0:nch, :])
                new_h5file[key_i].create_dataset('phase', data = phase)
                new_h5file[key_i].create_dataset('mag', data = mag)
            elif key_j == 'freq':
                new_h5file[key_i].create_dataset(key_j, data = h5data[key_i][key_j][:])
                
    
    new_h5file.close()
    
# old version assumes that there is a specific correction for each radius
def CorrectMagPhase(h5data, h5correction, name):
    
    new_h5file = h5py.File(name, 'w')
    try:
        
        data = h5py.File(h5data, 'r')
        correction = h5py.File(h5correction, 'r')
        
        for i, key_i in enumerate(data.keys()):
            
            new_h5file.create_group(key_i)
            
            for j, key_j in enumerate(data[key_i]):
                
                if key_j == 'mag':
                    correction_array = correction[key_i]['amplitude'][:]
                    new_h5file[key_i].create_dataset(key_j, data=correction_array.reshape(1, *correction_array.shape) * data[key_i][key_j][:])
                elif key_j =='phase':
                    correction_array = correction[key_i]['phase'][:]
                    new_h5file[key_i].create_dataset(key_j, data=correction_array.reshape(1, *correction_array.shape) + data[key_i][key_j][:])
                elif key_j =='freq':
                    new_h5file[key_i].create_dataset(key_j, data=data[key_i][key_j][:])
                    
        new_h5file.close()
        
    except BaseException as err:
        new_h5file.close()
        print(err)
        
# new version uses only a single mag and phase correction for the whole data
def CorrectMagPhase_old(h5data, h5correction, name):
    
    new_h5file = h5py.File(name, 'w')
    try:
        
        data = h5py.File(h5data, 'r')
        correction = h5py.File(h5correction, 'r')
        
        for i, key_i in enumerate(data.keys()):
            
            new_h5file.create_group(key_i)
            
            for j, key_j in enumerate(data[key_i]):
                
                if key_j == 'mag':
                    correction_array = correction[key_i]['amplitude'][:]
                    new_h5file[key_i].create_dataset(key_j, data=correction_array * data[key_i][key_j][:])
                elif key_j =='phase':
                    correction_array = correction[key_i]['phase'][:]
                    new_h5file[key_i].create_dataset(key_j, data=correction_array + data[key_i][key_j][:])
                elif key_j =='freq':
                    new_h5file[key_i].create_dataset(key_j, data=data[key_i][key_j][:])
                    
        new_h5file.close()
        
    except BaseException as err:
        new_h5file.close()
        print(err)
    
    
def SpatialPhaseCorrection(rad, init_phase):
    
    array_rad = 0.10
    rotation_angles = np.arange(0, 360, 3)
    initial_phase = - np.radians(init_phase)
    frequency = 25.8e9
    c = 3e8
    wavelength = c / frequency

    x = rad 
    y = 0

    x_array = array_rad * np.cos(np.radians(rotation_angles) + initial_phase)
    y_array = array_rad * np.sin(np.radians(rotation_angles) + initial_phase)
    d = np.sqrt((x - x_array) ** 2 + (y - y_array) ** 2)

    phase = 2 * np.pi * d / wavelength

    return phase - phase[0]

    
    
        


    

In [ ]:
os.listdir(VNARESULTS)

In [ ]:
os.listdir(VNADATAPATH)

In [ ]:
os.listdir(VNACORRECTIONS)

In [ ]:
corr_file = os.path.join(VNACORRECTIONS, '220119_cres2_vna_correction.h5')
result_file = os.path.join(VNARESULTS, '211220_cres2_vna_trace_magphase.h5')

name = os.path.join(VNARESULTS, '220119_cres2_vna_trace_mag_phase_corrected.h5')

CorrectMagPhase(result_file, corr_file, name)

# plot axial/angle mag surface

In [ ]:
name = os.path.join(VNARESULTS, '211220_cres2_vna_trace_magphase.h5')

magphase_data = h5py.File(name, 'r')

In [ ]:
magphase_data.keys()

In [ ]:
z_array = np.linspace(-10, 10, 5)
ang_array = np.int32(np.linspace(0, 357, 120))
irad = 30
ifreq = 800

#frequencies = datafile[acq]['freq'][:]

ang_grid, z_grid = np.meshgrid(ang_array, z_array)

mag_data = np.zeros((z_grid.shape))

for i, z in enumerate(z_array):

    mag_data[i, :] = magphase_data[f'range3cm_step1mm_z{int(z)}mm']['mag'][irad, :, ifreq]

    
#print(z_grid)


In [ ]:

print(np.var(20 * np.log10(np.flip(mag_data, 0) ), axis=(0, 1)) ** (1/2))


In [ ]:

sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13, 8))

ax = fig.add_subplot(1,1,1)

img = ax.imshow(20 * np.log10(mag_data ), aspect='auto', cmap=cmap, extent=(0, 360, -10, 10))

cbar = fig.colorbar(img, label ='dB')

ax.set_xlabel('Angle')
ax.set_ylabel('10-Slot z position')
ax.set_title('S21 Pattern for Fake CRES #2')

name = '220110_cres2_example_s21_mag_pattern_10slot'
#plt.savefig(os.path.join(PATH, 'plots', 'moth', name))
#ax = fig.add_subplot(1,1,1, projection='3d')

#plt.plot_surface(ang_grid, z_grid, 20 * np.log10(mag_data ), cmap=cmap, edgecolor='k')


#ax.set_zlim3d(-70, -40)


# Compare corrected vs uncorrected data

In [ ]:
name = os.path.join(VNARESULTS, '211220_cres2_vna_trace_magphase.h5')

magphase_data = h5py.File(name, 'r')

name_corrected = os.path.join(VNARESULTS, '220119_cres2_vna_trace_mag_phase_corrected.h5')

magphase_data_corrected = h5py.File(name_corrected, 'r')

In [ ]:
z_array = [0]#np.linspace(0)
ang_array = np.int32(np.linspace(0, 357, 120))
iradius_array = np.arange(0, 31, 1)
ifreq = 800

#frequencies = datafile[acq]['freq'][:]

ang_grid, z_grid = np.meshgrid(ang_array, z_array)



corrected_variance = np.zeros(iradius_array.size)
uncorrected_variance = np.zeros(iradius_array.size)

#############################################
mag_data = np.zeros((z_grid.shape))
for irad in iradius_array:

    for i, z in enumerate(z_array):

        mag_data[i, :] = magphase_data[f'range3cm_step1mm_z{int(z)}mm']['mag'][irad, :, ifreq]

        
    uncorrected_variance[irad] = np.var(20 * np.log10(np.flip(mag_data, 0) ), axis=(0, 1)) ** (1/2)

#############################################
mag_data_corrected = np.zeros((z_grid.shape))
for irad in iradius_array:

    for i, z in enumerate(z_array):

        mag_data_corrected[i, :] = magphase_data_corrected[f'range3cm_step1mm_z{int(z)}mm']['mag'][irad, :, ifreq]

        
    corrected_variance[irad] = np.var(20 * np.log10(np.flip(mag_data_corrected, 0) ), axis=(0, 1)) ** (1/2)
    
#print(z_grid)


In [ ]:
plt.plot(uncorrected_variance)
plt.plot(corrected_variance)

In [ ]:
z_array = np.linspace(-10, 10, 5)
ang_array = np.int32(np.linspace(0, 357, 120))
iradius_array = np.arange(0, 31, 1)
ifreq = 800

#frequencies = datafile[acq]['freq'][:]

ang_grid, z_grid = np.meshgrid(ang_array, z_array)



corrected_variance = np.zeros(iradius_array.size)
uncorrected_variance = np.zeros(iradius_array.size)

mag_data = np.zeros((z_grid.shape))

for irad in iradius_array:

    for i, z in enumerate(z_array):

        mag_data[i, :] = magphase_data[f'range3cm_step1mm_z{int(z)}mm']['mag'][irad, :, ifreq]

        
    uncorrected_variance[irad] = np.var(20 * np.log10(np.flip(mag_data, 0) ), axis=(0, 1)) ** (1/2)
    
mag_data_corrected = np.zeros((z_grid.shape))
for irad in iradius_array:

    for i, z in enumerate(z_array):

        mag_data_corrected[i, :] = magphase_data_corrected[f'range3cm_step1mm_z{int(z)}mm']['mag'][irad, :, ifreq]

        
    corrected_variance[irad] = np.var(20 * np.log10(np.flip(mag_data_corrected, 0) ), axis=(0, 1)) ** (1/2)
    
#print(z_grid)
